In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from sklearn import metrics
from tqdm.notebook import tqdm

import mlflow

import csv
import json
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import tempfile

from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, balanced_accuracy_score

from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from torch.utils.data import Dataset, DataLoader, IterableDataset
from torchmetrics import AveragePrecision

from tqdm.notebook import tqdm, trange
import tempfile

In [240]:
torch.__version__

'1.10.1+cu111'

In [241]:
print(f"¿CUDA disponible? {torch.cuda.is_available()}")

¿CUDA disponible? True


## Exploración de datos
Cargamos los datos de entrenamiento para ver que forma tienen

In [256]:
#train_dataset = pd.read_json('./data/meli-challenge-2019/spanish.train.jsonl.gz',lines=True)


In [257]:
#train_dataset.head(4)

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,reliable,Casita Muñecas Barbies Pintadas,DOLLHOUSES,train,"[casita, muñecas, barbies, pintadas]","[50001, 2, 50000, 3]",0,632,4895280
1,spanish,unreliable,Neceser Cromado Holográfico,TOILETRY_BAGS,train,"[neceser, cromado, holográfico]","[6, 4, 5]",1,632,4895280
2,spanish,unreliable,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS,train,"[funda, asiento, medida, chevrolet]","[9, 7, 10, 8]",2,632,4895280
3,spanish,unreliable,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS,train,"[embrague, ford, focus, one]","[11, 13, 12, 14]",3,632,4895280


In [243]:
# train_dataset.language.value_counts()

In [244]:
# train_dataset.label_quality.value_counts()

## Dataset

Tomado de https://github.com/DiploDatos/AprendizajeProfundo/blob/master/3_datasets.ipynb


In [258]:
x = np.zeros(632)
x[1] = 1 - 1e-4
torch.Tensor(x)

tensor([0.0000, 0.9999, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [2]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None, key = 'title'):
        """
        path: Ubicación a los datos (comprimidos con gzip)
        key: Columna que vamos a usar para entrenar
        """
        self.dataset_path = path
        self.transform = transform
        self.key = key

    def __iter__(self):
        """
        Habilita un iterador sobre los datos
        """
        with gzip.open(self.dataset_path, "rt") as fh:
            for l in fh:
                data = json.loads(l)    
                
                # one-hot encoding like.
                index = int(data['target'])
                encoded_target = np.zeros(632)
                encoded_target[index] = 1

                item = {
                    "data": data[self.key],
#                     "target": data['target']
                    "target": encoded_target #data['target']
                }
                
                if self.transform:
                    yield self.transform(item)
                else:
                    yield item


In [3]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]
        
        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
#            "data": torch.LongTensor(data),
            "data": torch.FloatTensor(data), 
            "target": torch.FloatTensor(target)
            
        }

In [4]:
# Definimos una collate_fn que nos retorne todos los arreglos de la misma longitud
# data_len = train_dataset.data.apply(lambda v: len(v))
# max_len = data_len.max()


pad_to_len = PadSequences(max_length=20)

### Juntando todo
Tenemos
* Una clase con la responsabilidad de entregar datos, potencialmente preprocesandolos si hace falta
* Una función que transforma los datos leidos para que todos los valores tengan la misma longitud

A partir de esto, creamos dos instancia del `DataLoader`: Uno para cargar los datos de entrenamiento y otro para cargar los datos de _test_

In [5]:
train_dataset = MeLiChallengeDataset('./data/meli-challenge-2019/spanish.train.jsonl.gz', key = 'data')
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn = pad_to_len)

In [6]:
test_dataset = MeLiChallengeDataset('./data/meli-challenge-2019/spanish.validation.jsonl.gz', key = 'data') 
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn = pad_to_len)

## MLP

Tomamos el MLP de las clases y agregamos los parametros que nos interesan

* Capa de entrada: Tantas neuronas como tokens
* Capa de salida: Tantas neuronas como categorías (632)

In [7]:
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden):
        """
        input_size: Número de neuronas de entrada
        output_size: Número de neuronas de salida
        hidden: Lista con los numeros de capas ocultas
        """
        super().__init__()
        assert len(hidden) > 0
        
        self._name = str(input_size) +'_'+ "_".join(map(lambda i: str(i), hidden)) + '_' + str(output_size)
        
        neurons = [input_size]  + hidden 
        parts = []
        for idx, each in enumerate(neurons[:-1]):
            parts.append(nn.Linear(each, neurons[idx + 1]))
            parts.append(nn.ReLU())
            
        parts = parts + [nn.Linear(neurons[-1], output_size), nn.Sigmoid()]
        
        self.model = nn.Sequential(*parts)
    
    def forward(self, x: torch.Tensor):
        return self.model(x)
    
    def name(self):
        return "MLP_" + self._name
    

In [8]:
MLP(10, 2 , hidden = [16]).name()

'MLP_10_16_2'

In [9]:
MLP(20,632,[256])

MLP(
  (model): Sequential(
    (0): Linear(in_features=20, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=632, bias=True)
    (3): Sigmoid()
  )
)

### Armado paso a paso

In [10]:
# Funciones utilitarias para obtener los datos en batch
get_train_dataloader = lambda bs: DataLoader(train_dataset, batch_size=bs, shuffle=False, num_workers=0, collate_fn = pad_to_len)
get_test_dataloader = lambda bs: DataLoader(test_dataset, batch_size=bs, shuffle=False, num_workers=0, collate_fn = pad_to_len)

#### Verificamos la forma de la entrada, salida y la etiqueta

In [11]:
it = iter(train_dataloader)
e = it.next()
data, target = e['data'], e['target']
data, target

/tmp/ipykernel_1857091/1489816806.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  "target": torch.FloatTensor(target)


(tensor([[5.0001e+04, 2.0000e+00, 5.0000e+04, 3.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [6.0000e+00, 4.0000e+00, 5.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.0000e+00, 7.0000e+00, 1.0000e+01, 8.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.1000e+01, 1.3000e+01, 1.2000e+01, 1.4000e+01, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.000

In [12]:
data.size(), target.size() # batch_size = 4

(torch.Size([4, 20]), torch.Size([4, 632]))

In [13]:
m = MLP(20, 632,[256])
output = m(data)
output, target

(tensor([[0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.3482, 0.6162, 0.3838,  ..., 0.3217, 0.4873, 0.4492],
         [0.1376, 0.6017, 0.2552,  ..., 0.2127, 0.3714, 0.4205],
         [0.0710, 0.5903, 0.1210,  ..., 0.1555, 0.3135, 0.4006]],
        grad_fn=<SigmoidBackward0>),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [14]:
# Convertimos a la forma esperada
#cmp = lambda t: torch.Tensor([[x] for x in t])
cmp = lambda t: t #ID
actual = cmp(target)
actual

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

#### Verificamos que se pueda calcular la función de pérdida y el _score_

In [276]:
loss = nn.BCELoss()
loss_value = loss(output, actual)
loss_value

tensor(13.4385, grad_fn=<BinaryCrossEntropyBackward0>)

In [277]:
categories_from_tensor = lambda t: torch.max(t.data, 1)[1]
y_true = categories_from_tensor(target)
y_true.numpy()

array([0, 1, 2, 3])

In [278]:
predicted = categories_from_tensor(output)
predicted.numpy(), target.numpy() 

(array([  0, 533, 533, 545]),
 array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [279]:
balanced_accuracy_score(categories_from_tensor(target).numpy(), predicted.numpy())

/users/tnino/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1999: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.25

#### Loop de entrenamiento

In [21]:
def run_train(dataloader, model, optimizer, loss):
    assert model.training, "model should be in training mode"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Run train loop in ${device}")
    running_loss = []
    model.to(device)
    for idx, batch in enumerate(tqdm(dataloader)):
        optimizer.zero_grad()

        input_data = batch["data"].to(device)
        output = model(input_data)

        target_data = batch["target"].to(device)
        
        target = cmp(target_data).to(device)
                            
        loss_value = loss(output, target)
        loss_value.backward()

        optimizer.step()
        running_loss.append(loss_value.item())
    return running_loss

In [283]:
optimizer = optim.Adam(m.parameters(), lr=1e-3, weight_decay=1e-5)
loss = nn.BCELoss()
m.train()
for epoch in trange(1):
    run_train(test_dataloader, m, optimizer, loss)

  0%|          | 0/1 [00:00<?, ?it/s]

Run train loop in $cuda:0


0it [00:00, ?it/s]

KeyboardInterrupt: 

#### Loop de validación

In [22]:
def run_eval(dataloader, model, loss):
    assert not model.training, "model should not be in training mode"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Run eval loop in ${device}")
    running_loss = []
    targets = []
    predictions = []
                     
    model.to(device)
    for batch in tqdm(dataloader):
        input_data = batch["data"].to(device)
        output = model(input_data)
                    
        target = batch["target"]
        target_t = cmp(target).to(device)
        
        running_loss.append(
            loss(output, target_t).item()
        )
        
        predicted = categories_from_tensor(output)
        y_true = categories_from_tensor(target)

        targets.extend(y_true.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())
        
    return running_loss, targets, predictions


In [ ]:
loss = nn.BCELoss()
m.eval()

for epoch in trange(1):
    _, targets, predictions = run_eval(test_dataloader, m, loss)
    print(balanced_accuracy_score(targets, predictions))


---

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
def run_experiment(params, **kwargs):
    """
    Run a single experiment

    params: A list of (model, loss_fn, optimizer,num_epochs) tuples
    """
    for(build_model, build_loss, build_optimizer, nepochs, batch_sizes) in params:

        # Pasamos varios valores para epochs, ejecutamos un experimento para cada uno
        for epochs in nepochs:
            for bs in batch_sizes:
                
                print(f"Experiment epochs:{epochs}, batch-size:{bs}")
                train_dataloader = get_train_dataloader(bs)
                test_dataloader = get_test_dataloader(bs)
                

                with mlflow.start_run():
                    # Crea nuevos objetos para cada epoca
                    model = build_model()
                    optimizer = build_optimizer(model)
                    loss = build_loss()

                    exp_name = model.name() + "_e" + str(epochs) + "_b" + str(bs)
                    mlflow.set_experiment(exp_name)

                    mlflow.log_param("model_name", model.name())
                    mlflow.log_param("epochs", str(epochs))
                    mlflow.log_param("batch_size", str(bs))

                    model.to(device)
                    for epoch in trange(epochs):
                        model.train()
                        running_loss = run_train(train_dataloader, model, optimizer, loss)
                        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)

                        model.eval()
                        running_loss, targets, predictions = run_eval(test_dataloader, model, loss)
                        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
                        mlflow.log_metric("test_avp", balanced_accuracy_score(targets, predictions), epoch)

                    print(f"Save results. epochs:{epochs}, batch-size:{bs}")
                    with tempfile.TemporaryDirectory() as tmpdirname:
                        targets = []
                        predictions = []

                        for batch in tqdm(test_dataloader):
                            output = model(batch["data"].to(device))
                            
                            y_true = categories_from_tensor(batch["target"])
                            targets.extend(y_true.numpy())
                            
                            predicted = categories_from_tensor(output)
                            predictions.extend(predicted.squeeze().detach().cpu().numpy())

                        filename = "{}/{}_predictions.csv.gz".format(tmpdirname, exp_name)
                        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
                            filename, index=False
                        )
                        mlflow.log_artifact(filename)

In [23]:
output_size = 632

In [24]:
loss = lambda : nn.BCELoss()

def getOptimizer(model, lr = 1e-3, wd = 1e-5):
    return optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
                   
epochs = [1, 2, 4]
batch_sizes = [24, 48, 64]

In [ ]:
mlp_test_experiment = [(lambda : MLP(20, output_size, hidden = [256]), loss, getOptimizer, [1], [4])]
run_experiment(mlp_test_experiment)

In [18]:


mlp_single_layer = [(lambda : MLP(20, output_size, hidden = [256]), loss, getOptimizer, epochs, batch_sizes)]
mlp_multiple_layers = [(lambda : MLP(20, output_size, hidden = [40,64,256, 512]), loss, getOptimizer, epochs, batch_sizes)]

In [ ]:
run_experiment(mlp_single_layer)

In [19]:
run_experiment(mlp_multiple_layers)

Experiment epochs:1, batch-size:24


2022/10/23 10:28:16 INFO mlflow.tracking.fluent: Experiment with name 'MLP_20_40_64_256_512_632_e1_b24' does not exist. Creating a new experiment.


  0%|          | 0/1 [00:00<?, ?it/s]

NameError: name 'run_train' is not defined